In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
matrix_size = 25
num_epochs = 30
num_classes = 10 # D-E problemleri için

## Problemler:
Problem A:  Matriste rasgele konumlarda 2 nokta bulunmaktadır. Model bu 2 nokta arasındaki Öklid mesafesini çıktı olarak verecektir.

Problem B:  Matriste değişken sayıda (her girişte farklı sayıda olabilir) N adet (3- 10 arası) nokta (rasgele konumlarda) bulunmaktadır. Model bu noktalardan birbirine en yakın olan ikisi arasındaki Öklid mesafesini çıktı olarak verecektir.

Problem C:  Matriste değişken sayıda (her girişte farklı sayıda olabilir) N adet (3- 10 arası) nokta (rasgele konumlarda) bulunmaktadır. Model bu noktalardan birbirine en uzak olan ikisi arasındaki Öklid mesafesini çıktı olarak verecektir.

Problem D:  Matriste değişken sayıda (her girişte farklı sayıda olabilir) N adet (1- 10 arası) nokta (rasgele konumlarda) bulunmaktadır. Model nokta sayısını çıktı olarak verecektir.

Problem E:  Matriste değişken sayıda (her girişte farklı sayıda olabilir) ve değişken büyüklükte N adet (1-10 arası) kare (rasgele konumlarda) bulunmaktadır. Kareler kesişebilir / kesişmeyebilir. Model kare sayısını çıktı olarak verecektir.

# A

In [ ]:
def load_data(problem, split):
    data = np.load(f'data/{problem}_{split}.npz')
    X = data['X'].astype(np.float32)
    y = data['y'].astype(np.float32).reshape(-1,1)
    return X, y

class DatasetA(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).unsqueeze(1)
        self.y = torch.from_numpy(y)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Veri yükleme yaptığım kısım
X_train, y_train = load_data('A', 'train')  # 800 örnek
X_test,  y_test  = load_data('A', 'test')   # 200 örnek
train_ds = DatasetA(X_train, y_train)
test_ds  = DatasetA(X_test,  y_test)

In [ ]:
class CNN_A(nn.Module):
    def __init__(self, f1=16, f2=32, f3=64, hidden=64):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,  f1, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), # giriş, çıkış, kernel boyutu ve padding
            nn.Conv2d(f1, f2, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(f2, f3, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )                                                                # f3 tane 3x3 lük blok var
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f3 * 3 * 3, hidden), nn.ReLU(),           # gizli katman
            nn.Linear(hidden, 1)    # çıktı katmanı (1 tane output olacak çünkü regresyon problemi)
        )

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    total = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        total += criterion(model(xb), yb).item() * xb.size(0)
    return total / len(loader.dataset)

criterion = nn.MSELoss()

In [ ]:
def run_training(model, train_loader, test_loader, criterion, optimizer, epochs): # değerlendirme kriteri olarak MSE kullandım. optimizer olarak da ADAM kullandım
    train_losses, test_losses = [], []
    for epoch in range(1, epochs+1):
        model.train()                   # eğitim modunu açıyoruz
        total_train = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()       # her epochtan önce gradyanları sıfırlıyorum
            loss = criterion(model(xb), yb) # loss hesaplıyorum
            loss.backward()         # gradyanları hesaplıyorum
            optimizer.step()        # ağırlıkları güncelliyorum
            total_train += loss.item() * xb.size(0)      # loss ları topluyorum
        train_losses.append(total_train / len(train_loader.dataset))    # ortalama loss
        
        model.eval()
        total_test = 0.0
        with torch.no_grad():
            for xb, yb in test_loader:  #test set üzerinde değerlendirme yapıyorum gradyan yok
                xb, yb = xb.to(device), yb.to(device)
                total_test += criterion(model(xb), yb).item() * xb.size(0)
        test_losses.append(total_test / len(test_loader.dataset))
        print(f"Epoch {epoch:02d} - Train MSE: {train_losses[-1]:.4f}, Test MSE: {test_losses[-1]:.4f}")
    return train_losses, test_losses

# default ayarlar
default_lr = 1e-3
default_bs = 32

train_loader = DataLoader(train_ds, batch_size=default_bs, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=default_bs, shuffle=False)

model = CNN_A().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=default_lr)

train_losses, test_losses = run_training(model, train_loader, test_loader, criterion, optimizer, num_epochs)

# çiz
plt.figure()
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), test_losses,  label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('CNN Loss')
plt.legend()
plt.show()

In [ ]:
grid = {                                # grid search için parametreler. küçük sayılarda olduğu için iyi ama çarpımlar artıkça kötüleşir                   
    'lr': [1e-4, 1e-3, 1e-2],
    'batch_size': [16, 32, 64],
    'f1': [16, 32],
    'f2': [32, 64],
    'hidden': [32, 64]
}

results = []
criterion = nn.MSELoss()

total = len(grid['lr'])*len(grid['batch_size'])*len(grid['f1'])*len(grid['f2'])*len(grid['hidden']) # lr 3 , batch_size 3, f1 2, f2 2, hidden 2 
cnt = 0                                                                                             # 3*3*2*2*2 = 72 farklı kombinasyondan en iyisini seçecez
for lr in grid['lr']:
    for bs in grid['batch_size']:
        for f1 in grid['f1']:
            for f2 in grid['f2']:
                for hidden in grid['hidden']:
                    cnt += 1
                    print(f"Run {cnt}/{total}: lr={lr}, bs={bs}, f1={f1}, f2={f2}, hidden={hidden}")
                    # DataLoader
                    train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True)    # yeni değerlerle data loaderi ve altta modeli oluşturuyoruz
                    test_loader  = DataLoader(test_ds,  batch_size=bs, shuffle=False)
                    # model
                    net = CNN_A(f1=f1, f2=f2, f3=f2*2, hidden=hidden).to(device)
                    opt = optim.Adam(net.parameters(), lr=lr)   # optimizer olarak ADAM
                    # kısa eğitim (10 epoch)
                    run_training(net, train_loader, test_loader, criterion, opt, epochs=10)
                    mse = evaluate(net, test_loader, criterion)     # test set üzerinde MSE hesaplıyoruz
                    results.append({'lr':lr,'bs':bs,'f1':f1,'f2':f2,'hidden':hidden,'mse':mse})

# en iyi 5 ayar
best5 = sorted(results, key=lambda x: x['mse'])[:5]
print("Top 5 Hiperparametre :")
for r in best5:
    print(r)

In [ ]:
best = best5[0]
opt_lr    = best['lr']
opt_bs    = best['bs']
opt_f1    = best['f1']
opt_f2    = best['f2']
opt_hidden= best['hidden']
print(f"En başarılı kombinasyon: lr={opt_lr}, batch_size={opt_bs}, f1={opt_f1}, f2={opt_f2}, hidden={opt_hidden}")

# DataLoader
train_loader = DataLoader(train_ds, batch_size=opt_bs, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=opt_bs, shuffle=False)

# model ve optimizasyon
model_opt = CNN_A(f1=opt_f1, f2=opt_f2, f3=opt_f2*2, hidden=opt_hidden).to(device)
optimizer_opt = optim.Adam(model_opt.parameters(), lr=opt_lr)

# eğitim ve kayıpların hesaplanması
train_losses_opt, test_losses_opt = run_training(model_opt, train_loader, test_loader, criterion, optimizer_opt, num_epochs)

# Sonuçları çiz
plt.figure()
plt.plot(range(1, num_epochs+1), train_losses_opt, label='Train Loss Opt')
plt.plot(range(1, num_epochs+1), test_losses_opt,  label='Test Loss Opt')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('Optimize  Hiperparametrelerle Loss')
plt.legend()
plt.show()

In [ ]:
data_sizes = {'25%': int(0.25*len(train_ds)), '50%': int(0.5*len(train_ds)), '100%': len(train_ds)}
test_results = {}
for label, n_samples in data_sizes.items():
    print(f"\n{label} lik veri kümesi ({n_samples} örnek)")
    idx = np.random.permutation(len(train_ds))[:n_samples]
    X_sub = X_train[idx]
    y_sub = y_train[idx]
    sub_ds = DatasetA(X_sub, y_sub)
    tl = DataLoader(sub_ds, batch_size=opt_bs, shuffle=True)
    net = CNN_A(f1=opt_f1, f2=opt_f2, f3=opt_f2*2, hidden=opt_hidden).to(device)
    optm = optim.Adam(net.parameters(), lr=opt_lr)
    # Eğit
    for epoch in range(1, num_epochs+1):
        net.train()
        for xb, yb in tl:
            xb, yb = xb.to(device), yb.to(device)
            optm.zero_grad()
            loss = criterion(net(xb), yb)
            loss.backward()
            optm.step()
    # test
    mse_final = evaluate(net, DataLoader(test_ds, batch_size=opt_bs, shuffle=False), criterion)
    test_results[label] = mse_final
    print(f"-> Test MSE: {mse_final:.2f}")
# Sonuçları çiz\ nplt.figure()
plt.bar(list(test_results.keys()), list(test_results.values()))
plt.xlabel('Eğitim Veri Oranı')
plt.ylabel('Test MSE')
plt.title('CNN Test MSE vs Eğitim Veri Oranı')
plt.show()

In [ ]:
model_opt.eval()
preds, true_vals = [], []
with torch.no_grad():
    for xb, yb in DataLoader(test_ds, batch_size=opt_bs, shuffle=False):
        xb = xb.to(device)
        out = model_opt(xb).cpu().numpy().reshape(-1)
        preds.extend(out)           # tahminleri listeye ekleme
        true_vals.extend(yb.numpy().reshape(-1)) # gerçek değerleri listeye ekleme
preds = np.array(preds)
true_vals = np.array(true_vals)
errors = np.abs(preds - true_vals)

# en iyi ve en kötü 5 örnek indeksleri
best_idx = np.argsort(errors)[:5]
worst_idx = np.argsort(errors)[-5:]

#  görselleştirme fonksiyonu
def plot_examples(indices, title):
    print(title)
    fig, axs = plt.subplots(1, len(indices), figsize=(15,3))
    for ax, idx in zip(axs, indices):
        mat = X_test[idx]
        ax.imshow(mat, cmap='gray_r', interpolation='nearest')
        # Izgarayı ekle
        ax.set_xticks(np.arange(-0.5, matrix_size, 1), minor=True)
        ax.set_yticks(np.arange(-0.5, matrix_size, 1), minor=True)
        ax.grid(which='minor', color='black', linestyle='-', linewidth=0.5)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f"True: {true_vals[idx]:.2f}\nPred: {preds[idx]:.2f}\nErr: {errors[idx]:.2f}")
    plt.tight_layout()
    plt.show()

plot_examples(best_idx, "En İyi Tahminler (Düşük Error)")
plot_examples(worst_idx, "En Kötü Tahminler (Yüksek Error)")


# B

In [ ]:
def load_data(problem, split):
    data = np.load(f'data/{problem}_{split}.npz')
    X = data['X'].astype(np.float32) 
    y = data['y'].astype(np.float32).reshape(-1,1)
    return X, y

class DatasetB(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).unsqueeze(1)
        self.y = torch.from_numpy(y)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

X_train, y_train = load_data('B', 'train')  # 800 örnek
X_test,  y_test  = load_data('B', 'test')   # 200 örnek
train_ds = DatasetB(X_train, y_train)
test_ds  = DatasetB(X_test,  y_test)

In [ ]:
class CNN_B(nn.Module):
    def __init__(self, f1=16, f2=32, f3=64, hidden=64):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,  f1, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(f1, f2, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(f2, f3, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f3 * 3 * 3, hidden), nn.ReLU(),
            nn.Linear(hidden, 1)
        )
    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    total = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        total += criterion(model(xb), yb).item() * xb.size(0)
    return total / len(loader.dataset)

criterion = nn.MSELoss()

In [ ]:
def run_training(model, train_loader, test_loader, criterion, optimizer, epochs):
    train_losses, test_losses = [], []
    for epoch in range(1, epochs+1):
        model.train()
        t_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()
            t_loss += loss.item() * xb.size(0)
        train_losses.append(t_loss / len(train_loader.dataset))
        # test
        test_losses.append(evaluate(model, test_loader, criterion))
        print(f"Epoch {epoch:02d} - Train MSE: {train_losses[-1]:.4f}, Test MSE: {test_losses[-1]:.4f}")
    return train_losses, test_losses

# default ayarlar
lr_def = 1e-3
bs_def = 64
train_loader = DataLoader(train_ds, batch_size=bs_def, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=bs_def, shuffle=False)
model_def = CNN_B().to(device)
opt_def = optim.Adam(model_def.parameters(), lr=lr_def)

train_losses_def, test_losses_def = run_training(model_def, train_loader, test_loader, criterion, opt_def, num_epochs)

# Grafik
plt.figure()
plt.plot(range(1, num_epochs+1), train_losses_def, label='Train')
plt.plot(range(1, num_epochs+1), test_losses_def,  label='Test')
plt.xlabel('Epoch'); plt.ylabel('MSE')
plt.title('CNN_B'); plt.legend(); plt.show()

In [ ]:
grid = {
    'lr': [1e-4,1e-3,1e-2],
    'batch_size': [16,32,64],
    'f1': [16,32],
    'f2': [32,64],
    'hidden': [32,64]
}
results = []
cnt = 0
total = len(grid['lr'])*len(grid['batch_size'])*len(grid['f1'])*len(grid['f2'])*len(grid['hidden'])
for lr in grid['lr']:
    for bs in grid['batch_size']:
        for f1 in grid['f1']:
            for f2 in grid['f2']:
                for hid in grid['hidden']:
                    cnt += 1
                    print(f"Run {cnt}/{total}: lr={lr}, bs={bs}, f1={f1}, f2={f2}, hidden={hid}")
                    tl = DataLoader(train_ds, batch_size=bs, shuffle=True)
                    vl = DataLoader(test_ds,  batch_size=bs, shuffle=False)
                    net = CNN_B(f1=f1, f2=f2, f3=f2*2, hidden=hid).to(device)
                    opt = optim.Adam(net.parameters(), lr=lr)
                    run_training(net, tl, vl, criterion, opt, epochs=10)
                    mse = evaluate(net, vl, criterion)
                    results.append({'lr':lr,'bs':bs,'f1':f1,'f2':f2,'hidden':hid,'mse':mse})
# Top5
best5 = sorted(results, key=lambda x: x['mse'])[:5]
print("Top 5 CNN_B parametreleri:")
for r in best5: print(r)

In [ ]:
best = best5[0]
lr_opt = best['lr']; bs_opt = best['bs']
f1_opt = best['f1']; f2_opt = best['f2']; hid_opt = best['hidden']
print(f"En başarılı kombinasyon: lr={lr_opt}, bs={bs_opt}, f1={f1_opt}, f2={f2_opt}, hidden={hid_opt}")
train_loader = DataLoader(train_ds, batch_size=bs_opt, shuffle=True)
# test loader remains
test_loader = DataLoader(test_ds,  batch_size=bs_opt, shuffle=False)
model_opt = CNN_B(f1=f1_opt,f2=f2_opt,f3=f2_opt*2,hidden=hid_opt).to(device)
opt_opt = optim.Adam(model_opt.parameters(), lr=lr_opt)
train_losses_opt, test_losses_opt = run_training(model_opt, train_loader, test_loader, criterion, opt_opt, num_epochs)
# Grafik
plt.figure()
plt.plot(range(1, num_epochs+1), train_losses_opt, label='Train Opt')
plt.plot(range(1, num_epochs+1), test_losses_opt,  label='Test Opt')
plt.xlabel('Epoch'); plt.ylabel('MSE')
plt.title('Optimize  Hiperparametrelerle Loss'); plt.legend(); plt.show()

In [ ]:
data_sizes = {'25%':int(0.25*len(train_ds)),'50%':int(0.5*len(train_ds)),'100%':len(train_ds)}
res = {}
for label,n in data_sizes.items():
    print(f"{label} lik veri kümesi({n} örnek)")
    idx = np.random.permutation(len(train_ds))[:n]
    X_sub = X_train[idx]; y_sub = y_train[idx]
    sub_ds = DatasetB(X_sub,y_sub)
    tl = DataLoader(sub_ds, batch_size=bs_opt, shuffle=True)
    net= CNN_B(f1=f1_opt,f2=f2_opt,f3=f2_opt*2,hidden=hid_opt).to(device)
    optm= optim.Adam(net.parameters(), lr=lr_opt)
    for epoch in range(1,num_epochs+1):
        net.train()
        for xb,yb in tl:
            xb,yb=xb.to(device),yb.to(device)
            optm.zero_grad(); loss=criterion(net(xb),yb); loss.backward(); optm.step()
    res[label] = evaluate(net, DataLoader(test_ds,batch_size=bs_opt), criterion)
    print(f"-> Test MSE: {res[label]:.4f}")
plt.figure(); plt.bar(res.keys(), res.values()); plt.xlabel('Veri Oranı'); plt.ylabel('Test MSE'); plt.title('CNN_B Test MSE vs Veri Oranı'); plt.show()


In [ ]:
model_opt.eval(); preds, trues = [], []
with torch.no_grad():
    for xb,yb in DataLoader(test_ds, batch_size=bs_opt):
        xb=xb.to(device)
        out=model_opt(xb).cpu().numpy().reshape(-1)
        preds.extend(out); trues.extend(yb.numpy().reshape(-1))
preds=np.array(preds); trues=np.array(trues)
errors=np.abs(preds-trues)

# Histogram
plt.figure(); plt.hist(errors, bins=30); plt.xlabel('Hata'); plt.ylabel('Frekans'); plt.title('Error Histogram'); plt.show()
# Hata vs True
plt.figure(); plt.scatter(trues, errors, alpha=0.3); plt.xlabel('True'); plt.ylabel('Error'); plt.title('Error vs True'); plt.show()
# En iyi/kötü örnekler
best_idx=np.argsort(errors)[:5]; worst_idx=np.argsort(errors)[-5:]

def plot_examples(indices, title):
    print(title)
    fig,axs=plt.subplots(1,len(indices),figsize=(15,3))
    for ax,idx in zip(axs,indices):
        mat=X_test[idx]
        ax.imshow(mat,cmap='gray_r',interpolation='nearest')
        ax.set_xticks(np.arange(-0.5,matrix_size,1),minor=True)
        ax.set_yticks(np.arange(-0.5,matrix_size,1),minor=True)
        ax.grid(which='minor',color='black',linestyle='-',linewidth=0.5)
        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title(f"T:{trues[idx]:.2f}\nP:{preds[idx]:.2f}\nE:{errors[idx]:.2f}")
    plt.tight_layout(); plt.show()

plot_examples(best_idx, 'En İyi Tahminler')
plot_examples(worst_idx, 'En Kötü Tahminler')


# C

In [ ]:
def load_data(problem, split):
    data = np.load(f'data/{problem}_{split}.npz')
    X = data['X'].astype(np.float32)
    y = data['y'].astype(np.float32).reshape(-1,1)
    return X, y

class DatasetC(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).unsqueeze(1)
        self.y = torch.from_numpy(y)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

X_train, y_train = load_data('C', 'train')  # 800 örnek
X_test,  y_test  = load_data('C', 'test')   # 200 örnek
train_ds = DatasetC(X_train, y_train)
test_ds  = DatasetC(X_test,  y_test)

In [ ]:
class CNN_C(nn.Module):
    def __init__(self, f1=16, f2=32, f3=64, hidden=64):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,  f1, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(f1, f2, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(f2, f3, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f3 * 3 * 3, hidden), nn.ReLU(),
            nn.Linear(hidden, 1)
        )
    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    total = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        total += criterion(model(xb), yb).item() * xb.size(0)
    return total / len(loader.dataset)

criterion = nn.MSELoss()

In [ ]:
def run_training(model, train_loader, test_loader, criterion, optimizer, epochs):
    train_losses, test_losses = [], []
    for epoch in range(1, epochs+1):
        model.train()
        t_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()
            t_loss += loss.item() * xb.size(0)
        train_losses.append(t_loss / len(train_loader.dataset))
        test_losses.append(evaluate(model, test_loader, criterion))
        print(f"Epoch {epoch:02d} - Train MSE: {train_losses[-1]:.4f}, Test MSE: {test_losses[-1]:.4f}")
    return train_losses, test_losses

# Default ayarlar
lr_def = 1e-3
bs_def = 32
train_loader = DataLoader(train_ds, batch_size=bs_def, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=bs_def, shuffle=False)
model_def = CNN_C().to(device)
opt_def = optim.Adam(model_def.parameters(), lr=lr_def)

train_losses_def, test_losses_def = run_training(model_def, train_loader, test_loader, criterion, opt_def, num_epochs)

# Grafik
plt.figure()
plt.plot(range(1, num_epochs+1), train_losses_def, label='Train')
plt.plot(range(1, num_epochs+1), test_losses_def,  label='Test')
plt.xlabel('Epoch'); plt.ylabel('MSE')
plt.title('CNN_C'); plt.legend(); plt.show()


In [ ]:
grid = {
    'lr': [1e-4,1e-3,1e-2],
    'batch_size': [16,32,64],
    'f1': [16,32],
    'f2': [32,64],
    'hidden': [32,64]
}
results = []
cnt = 0
total = len(grid['lr'])*len(grid['batch_size'])*len(grid['f1'])*len(grid['f2'])*len(grid['hidden'])
for lr in grid['lr']:
    for bs in grid['batch_size']:
        for f1 in grid['f1']:
            for f2 in grid['f2']:
                for hid in grid['hidden']:
                    cnt += 1
                    print(f"Run {cnt}/{total}: lr={lr}, bs={bs}, f1={f1}, f2={f2}, hidden={hid}")
                    tl = DataLoader(train_ds, batch_size=bs, shuffle=True)
                    vl = DataLoader(test_ds,  batch_size=bs, shuffle=False)
                    net = CNN_C(f1=f1, f2=f2, f3=f2*2, hidden=hid).to(device)
                    opt = optim.Adam(net.parameters(), lr=lr)
                    run_training(net, tl, vl, criterion, opt, epochs=10)
                    mse = evaluate(net, vl, criterion)
                    results.append({'lr':lr,'bs':bs,'f1':f1,'f2':f2,'hidden':hid,'mse':mse})
# Top5
best5 = sorted(results, key=lambda x: x['mse'])[:5]
print("Top 5 CNN_C Hiperparametreleri:")
for r in best5: print(r)


In [ ]:
best = best5[0]
lr_opt = best['lr']; bs_opt = best['bs']
f1_opt = best['f1']; f2_opt = best['f2']; hid_opt = best['hidden']
print(f"En başarılı kombinasyon: lr={lr_opt}, bs={bs_opt}, f1={f1_opt}, f2={f2_opt}, hidden={hid_opt}")
train_loader = DataLoader(train_ds, batch_size=bs_opt, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=bs_opt, shuffle=False)
model_opt = CNN_C(f1=f1_opt,f2=f2_opt,f3=f2_opt*2,hidden=hid_opt).to(device)
opt_opt = optim.Adam(model_opt.parameters(), lr=lr_opt)
train_losses_opt, test_losses_opt = run_training(model_opt, train_loader, test_loader, criterion, opt_opt, num_epochs)
# Grafik
plt.figure()
plt.plot(range(1, num_epochs+1), train_losses_opt, label='Train Opt')
plt.plot(range(1, num_epochs+1), test_losses_opt,  label='Test Opt')
plt.xlabel('Epoch'); plt.ylabel('MSE')
plt.title('Optimize  Hiperparametrelerle Loss'); plt.legend(); plt.show()

In [ ]:
data_sizes = {'25%':int(0.25*len(train_ds)),'50%':int(0.5*len(train_ds)),'100%':len(train_ds)}
res = {}
for label,n in data_sizes.items():
    print(f"{label} lik veri kümesi ({n} örnek)")
    idx = np.random.permutation(len(train_ds))[:n]
    X_sub = X_train[idx]; y_sub = y_train[idx]
    sub_ds = DatasetC(X_sub,y_sub)
    tl = DataLoader(sub_ds, batch_size=bs_opt, shuffle=True)
    net= CNN_C(f1=f1_opt,f2=f2_opt,f3=f2_opt*2,hidden=hid_opt).to(device)
    optm= optim.Adam(net.parameters(), lr=lr_opt)
    for epoch in range(1,num_epochs+1):
        net.train()
        for xb,yb in tl:
            xb,yb=xb.to(device),yb.to(device)
            optm.zero_grad(); loss=criterion(net(xb),yb); loss.backward(); optm.step()
    res[label] = evaluate(net, DataLoader(test_ds,batch_size=bs_opt), criterion)
    print(f"{label} -> Test MSE: {res[label]:.4f}")
plt.figure(); plt.bar(res.keys(), res.values()); plt.xlabel('Veri Oranı'); plt.ylabel('Test MSE'); plt.title('CNN_C Test MSE vs Veri Oranı'); plt.show()


In [ ]:
model_opt.eval(); preds,trues=[],[]
with torch.no_grad():
    for xb,yb in DataLoader(test_ds,batch_size=bs_opt):
        xb=xb.to(device)
        out=model_opt(xb).cpu().numpy().reshape(-1)
        preds.extend(out); trues.extend(yb.numpy().reshape(-1))
preds=np.array(preds); trues=np.array(trues)
errors=np.abs(preds-trues)

# Histogram
plt.figure(); plt.hist(errors, bins=30); plt.xlabel('Hata'); plt.ylabel('Frekans'); plt.title('Error Histogram'); plt.show()
# Error vs True
plt.figure(); plt.scatter(trues, errors, alpha=0.3); plt.xlabel('True'); plt.ylabel('Error'); plt.title('Error vs True'); plt.show()
# En iyi/kötü örnekler
best_idx=np.argsort(errors)[:5]; worst_idx=np.argsort(errors)[-5:]

def plot_example(mat, true, pred, err, title_prefix=""):
    fig, ax = plt.subplots(figsize=(3,3))
    # 1) Matrisin kendisini çiz
    ax.imshow(mat, cmap='gray_r', interpolation='nearest')
    # 2) Hücre sınırları için ızgara
    ax.set_xticks(np.arange(-.5, 25, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 25, 1), minor=True)
    ax.grid(which='minor', color='black', linewidth=0.5)
    ax.tick_params(which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
    # 3) Noktaları üzerine koy (mat==1 olan koordinatlar)
    ys, xs = np.where(mat == 1)
    ax.scatter(xs, ys, s=100, facecolors='none', edgecolors='red', linewidths=2)
    # 4) Başlık
    ax.set_title(
        f"{title_prefix}\nTrue: {true:.2f}  Pred: {pred:.2f}\nErr: {err:.2f}",
        fontsize=8
    )
    plt.tight_layout()
    plt.show()

plot_examples(best_idx, 'Best Predictions')
plot_examples(worst_idx, 'Worst Predictions')

# D

In [ ]:
def load_data(problem, split):
    data = np.load(f'data/{problem}_{split}.npz')
    X = data['X'].astype(np.float32)
    y = data['y'].astype(np.int64).reshape(-1) - 1  # sınıflar 0..9
    return X, y

class DatasetD(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).unsqueeze(1)
        self.y = torch.from_numpy(y)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

X_train, y_train = load_data('D', 'train')
X_test,  y_test  = load_data('D', 'test')
train_ds = DatasetD(X_train, y_train)
test_ds  = DatasetD(X_test,  y_test)


In [ ]:
class CNN_D(nn.Module):
    def __init__(self, f1=16, f2=32, f3=64, hidden=64):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,  f1, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(f1, f2, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(f2, f3, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f3 * 3 * 3, hidden), nn.ReLU(),
            nn.Linear(hidden, num_classes)
        )
    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

# eğitim ve değerlendirme fonksiyonları - artık değerlendirme kriteri olarak accuracy hesaplayacaz
def train_epoch(model, loader, criterion, optimizer):
    model.train(); total_loss = 0;
    correct = 0
    for xb,yb in loader:
        xb,yb=xb.to(device), yb.to(device)
        optimizer.zero_grad(); out=model(xb); loss=criterion(out,yb)
        loss.backward(); optimizer.step(); total_loss+=loss.item()*xb.size(0)
        pred=out.argmax(dim=1); correct+= (pred==yb).sum().item()
    return total_loss/len(loader.dataset), correct/len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval(); total_loss=0; correct=0
    for xb,yb in loader:
        xb,yb=xb.to(device), yb.to(device)
        out=model(xb); total_loss+=criterion(out,yb).item()*xb.size(0)
        correct+= (out.argmax(dim=1)==yb).sum().item()
    return total_loss/len(loader.dataset), correct/len(loader.dataset)

In [ ]:
#default ayarlar
lr_def=1e-3; bs_def=32

train_loader=DataLoader(train_ds, batch_size=bs_def, shuffle=True)
test_loader=DataLoader(test_ds,  batch_size=bs_def, shuffle=False)
model_def=CNN_D().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model_def.parameters(), lr=lr_def)

train_losses, train_accs = [], []
test_losses, test_accs = [], []
for epoch in range(1, num_epochs+1):
    tr_loss, tr_acc = train_epoch(model_def, train_loader, criterion, optimizer)
    te_loss, te_acc = evaluate(model_def, test_loader, criterion)
    train_losses.append(tr_loss); train_accs.append(tr_acc)
    test_losses.append(te_loss); test_accs.append(te_acc)
    print(f"Epoch {epoch:02d} - Train Acc: {tr_acc:.3f}, Test Acc: {te_acc:.3f}, Train Loss: {tr_loss:.3f}, Test Loss: {te_loss:.3f}")

# Grafik: Accuracy
plt.figure(); plt.plot(range(1,num_epochs+1), train_accs, label='Train Acc'); plt.plot(range(1,num_epochs+1), test_accs, label='Test Acc')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('CNN_D'); plt.legend(); plt.show()

In [ ]:
grid={ 'lr':[1e-4,1e-3,1e-2], 'batch_size':[16,32,64], 'f1':[16,32], 'f2':[32,64], 'hidden':[32,64] }
results=[]; cnt=0; total=len(grid['lr'])*len(grid['batch_size'])*len(grid['f1'])*len(grid['f2'])*len(grid['hidden'])
for lr in grid['lr']:
    for bs in grid['batch_size']:
        for f1 in grid['f1']:
            for f2 in grid['f2']:
                for hid in grid['hidden']:
                    cnt+=1; print(f"Run {cnt}/{total}: lr={lr}, bs={bs}, f1={f1}, f2={f2}, hid={hid}")
                    tl=DataLoader(train_ds,batch_size=bs,shuffle=True)
                    vl=DataLoader(test_ds,batch_size=bs,shuffle=False)
                    net=CNN_D(f1=f1,f2=f2,f3=f2*2,hidden=hid).to(device)
                    opt=optim.Adam(net.parameters(), lr=lr)
                    for _ in range(10): train_epoch(net, tl, criterion, opt)
                    _, acc = evaluate(net, vl, criterion)
                    results.append({'lr':lr,'bs':bs,'f1':f1,'f2':f2,'hidden':hid,'acc':acc})
best5=sorted(results, key=lambda x: -x['acc'])[:5]
print("Top 5 CNN_D Hiperparametreleri:")
for r in best5: print(r)



In [ ]:
best=best5[0]
lr_opt=best['lr']; bs_opt=best['bs']; f1_opt=best['f1']; f2_opt=best['f2']; hid_opt=best['hidden']
print(f"En iyi kombinasyon: lr={lr_opt}, bs={bs_opt}, f1={f1_opt}, f2={f2_opt}, hidden={hid_opt}")
train_loader=DataLoader(train_ds,batch_size=bs_opt,shuffle=True)
test_loader=DataLoader(test_ds,batch_size=bs_opt,shuffle=False)
model_opt=CNN_D(f1=f1_opt,f2=f2_opt,f3=f2_opt*2,hidden=hid_opt).to(device)
opt=optim.Adam(model_opt.parameters(), lr=lr_opt)
train_accs_opt, test_accs_opt = [], []
for epoch in range(1, num_epochs+1):
    _, tr_acc = train_epoch(model_opt, train_loader, criterion, opt)
    _, te_acc = evaluate(model_opt, test_loader, criterion)
    train_accs_opt.append(tr_acc); test_accs_opt.append(te_acc)
    print(f"Epoch {epoch:02d} - Train Acc: {tr_acc:.3f}, Test Acc: {te_acc:.3f}")
plt.figure(); plt.plot(range(1,num_epochs+1), train_accs_opt, label='Train Acc Opt'); plt.plot(range(1,num_epochs+1), test_accs_opt, label='Test Acc Opt')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Optimize  Hiperparametrelerle Acc'); plt.legend(); plt.show()

In [ ]:
data_sizes={'25%':int(0.25*len(train_ds)),'50%':int(0.5*len(train_ds)),'100%':len(train_ds)}
acc_res={}
for label,n in data_sizes.items():
    print(f"{label} lik veri kümesi ({n} örnek)")
    idx=np.random.permutation(len(train_ds))[:n]
    X_sub=X_train[idx]; y_sub=y_train[idx]
    sub_ds=DatasetD(X_sub,y_sub)
    tl=DataLoader(sub_ds,batch_size=bs_opt,shuffle=True)
    net=CNN_D(f1=f1_opt,f2=f2_opt,f3=f2_opt*2,hidden=hid_opt).to(device)
    optm=optim.Adam(net.parameters(), lr=lr_opt)
    for _ in range(num_epochs): train_epoch(net, tl, criterion, optm)
    _, acc_res[label] = evaluate(net, DataLoader(test_ds,batch_size=bs_opt), criterion)
    print(f"-> Test Acc: {acc_res[label]:.3f}")
plt.figure(); plt.bar(acc_res.keys(), acc_res.values()); plt.xlabel('Veri Oranı'); plt.ylabel('Test Acc'); plt.title('CNN_D Test Acc vs Veri Oranı'); plt.show()


In [ ]:
model_opt.eval()
r_preds, r_trues = [], []
with torch.no_grad():
    for xb,yb in DataLoader(test_ds,batch_size=bs_opt):
        xb=xb.to(device)
        out=model_opt(xb).cpu().numpy()
        preds=np.argmax(out,axis=1)
        r_preds.extend(preds); r_trues.extend(yb.numpy())
r_preds=np.array(r_preds); r_trues=np.array(r_trues)
correct_idx = np.where(r_preds==r_trues)[0][:5]
wrong_idx   = np.where(r_preds!=r_trues)[0][:5]

def plot_class_examples(indices, title):
    print(title)
    fig,axs=plt.subplots(1,len(indices),figsize=(15,3))
    for ax,idx in zip(axs, indices):
        mat = X_test[idx]
        ax.imshow(mat, cmap='gray_r', interpolation='nearest')
        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title(f"True:{r_trues[idx]+1}\nPred:{r_preds[idx]+1}")
    plt.tight_layout(); plt.show()

plot_class_examples(correct_idx, 'Doğru Sınıflandırılanlar')
plot_class_examples(wrong_idx,   'Yanlış Sınıflandırılanlar')

# E

In [ ]:
def load_data(problem, split):
    data = np.load(f'data/{problem}_{split}.npz')
    X = data['X'].astype(np.float32)
    y = data['y'].astype(np.int64).reshape(-1) - 1   # sınıf etiketleri 0..9
    return X, y

class DatasetE(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).unsqueeze(1)
        self.y = torch.from_numpy(y)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

X_train, y_train = load_data('E', 'train')
X_test,  y_test  = load_data('E', 'test')
train_ds = DatasetE(X_train, y_train)
test_ds  = DatasetE(X_test,  y_test)

In [ ]:
class CNN_E(nn.Module):
    def __init__(self, f1=16, f2=32, f3=64, hidden=64):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,  f1, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(f1, f2, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(f2, f3, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f3 * 3 * 3, hidden), nn.ReLU(),
            nn.Linear(hidden, num_classes)
        )
    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

# eğitim ve değerlendirme fonksiyonları
def train_epoch(model, loader, criterion, optimizer):
    model.train(); total_loss=0; correct=0
    for xb, yb in loader:
        xb,yb = xb.to(device), yb.to(device)
        optimizer.zero_grad(); out = model(xb);
        loss = criterion(out, yb); loss.backward(); optimizer.step()
        total_loss += loss.item() * xb.size(0)
        pred = out.argmax(dim=1); correct += (pred==yb).sum().item()
    return total_loss/len(loader.dataset), correct/len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval(); total_loss=0; correct=0
    for xb, yb in loader:
        xb,yb = xb.to(device), yb.to(device)
        out = model(xb); total_loss += criterion(out,yb).item()*xb.size(0)
        correct += (out.argmax(dim=1)==yb).sum().item()
    return total_loss/len(loader.dataset), correct/len(loader.dataset)


In [ ]:
#default ayarlar
lr_def=1e-3; bs_def=32

train_loader = DataLoader(train_ds, batch_size=bs_def, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=bs_def, shuffle=False)
model_def = CNN_E().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_def.parameters(), lr=lr_def)

train_losses, train_accs = [], []
test_losses, test_accs   = [], []
for epoch in range(1, num_epochs+1):
    tr_loss, tr_acc = train_epoch(model_def, train_loader, criterion, optimizer)
    te_loss, te_acc = evaluate(model_def, test_loader, criterion)
    train_losses.append(tr_loss); train_accs.append(tr_acc)
    test_losses.append(te_loss); test_accs.append(te_acc)
    print(f"Epoch {epoch:02d} - Train Acc: {tr_acc:.3f}, Test Acc: {te_acc:.3f}")
# Grafik: Accuracy
plt.figure(); plt.plot(train_accs, label='Train'); plt.plot(test_accs, label='Test')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('CNN_E'); plt.legend(); plt.show()

In [ ]:
grid = {
    'lr': [1e-4,1e-3,1e-2],
    'batch_size': [16,32,64],
    'f1': [16,32],
    'f2': [32,64],
    'hidden': [32,64]
}
results=[]; cnt=0
total = len(grid['lr'])*len(grid['batch_size'])*len(grid['f1'])*len(grid['f2'])*len(grid['hidden'])
for lr in grid['lr']:
    for bs in grid['batch_size']:
        for f1 in grid['f1']:
            for f2 in grid['f2']:
                for hid in grid['hidden']:
                    cnt+=1; print(f"Run {cnt}/{total}")
                    tl = DataLoader(train_ds,batch_size=bs,shuffle=True)
                    vl = DataLoader(test_ds, batch_size=bs,shuffle=False)
                    net=CNN_E(f1=f1,f2=f2,f3=f2*2,hidden=hid).to(device)
                    opt=optim.Adam(net.parameters(), lr=lr)
                    for _ in range(10): train_epoch(net, tl, criterion, opt)
                    _, acc = evaluate(net, vl, criterion)
                    results.append({'lr':lr,'bs':bs,'f1':f1,'f2':f2,'hidden':hid,'acc':acc})
best5 = sorted(results, key=lambda x: -x['acc'])[:5]
print("Top 5 CNN_E hiperparametreleri:")
for r in best5: print(r)

In [ ]:
best = best5[1]
lr_opt, bs_opt = best['lr'], best['bs']
f1_opt, f2_opt, hid_opt = best['f1'], best['f2'], best['hidden']
print(f"En iyi kombinasyon: lr={lr_opt}, bs={bs_opt}, f1={f1_opt}, f2={f2_opt}, hidden={hid_opt}")
train_loader = DataLoader(train_ds, batch_size=bs_opt, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=bs_opt, shuffle=False)
model_opt = CNN_E(f1=f1_opt,f2=f2_opt,f3=f2_opt*2,hidden=hid_opt).to(device)
optimizer = optim.Adam(model_opt.parameters(), lr=lr_opt)
train_accs_opt, test_accs_opt = [], []
for epoch in range(1, num_epochs+1):
    _, tr_acc = train_epoch(model_opt, train_loader, criterion, optimizer)
    _, te_acc = evaluate(model_opt, test_loader, criterion)
    train_accs_opt.append(tr_acc); test_accs_opt.append(te_acc)
    print(f"Epoch {epoch:02d} - Train Acc: {tr_acc:.3f}, Test Acc: {te_acc:.3f}")
plt.figure(); plt.plot(train_accs_opt, label='Train'); plt.plot(test_accs_opt, label='Test')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Optimize  Hiperparametrelerle Acc'); plt.legend(); plt.show()

In [ ]:
data_sizes={'25%':int(0.25*len(train_ds)),'50%':int(0.5*len(train_ds)),'100%':len(train_ds)}
acc_res={}
for label,n in data_sizes.items():
    idx=np.random.permutation(len(train_ds))[:n]
    sub_ds=DatasetE(X_train[idx], y_train[idx])
    tl=DataLoader(sub_ds,batch_size=bs_opt,shuffle=True)
    net=CNN_E(f1=f1_opt,f2=f2_opt,f3=f2_opt*2,hidden=hid_opt).to(device)
    optm=optim.Adam(net.parameters(), lr=lr_opt)
    for _ in range(num_epochs): train_epoch(net, tl, criterion, optm)
    _, acc_res[label] = evaluate(net, DataLoader(test_ds,batch_size=bs_opt), criterion)
    print(f"{label} -> Test Acc: {acc_res[label]:.3f}")
plt.figure(); plt.bar(acc_res.keys(), acc_res.values()); plt.xlabel('Veri Oranı'); plt.ylabel('Accuracy'); plt.title('CNN_E Test Acc vs Veri Oranı'); plt.show()

In [ ]:
model_opt.eval(); preds, trues = [], []
with torch.no_grad():
    for xb,yb in DataLoader(test_ds, batch_size=bs_opt):
        xb=xb.to(device)
        out=model_opt(xb).cpu().numpy()
        p=np.argmax(out,axis=1)
        preds.extend(p); trues.extend(yb.numpy())
preds, trues = np.array(preds), np.array(trues)
correct_idx = np.where(preds==trues)[0][:5]
wrong_idx   = np.where(preds!=trues)[0][:5]

def plot_examples(indices, title):
    print(title)
    fig,axs=plt.subplots(1,len(indices),figsize=(15,3))
    for ax,idx in zip(axs,indices):
        mat=X_test[idx]
        ax.imshow(mat,cmap='gray_r',interpolation='nearest')
        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title(f"True:{trues[idx]+1}\nPred:{preds[idx]+1}")
    plt.tight_layout(); plt.show()

plot_examples(correct_idx, 'Doğru Sınıflandırılanlar')
plot_examples(wrong_idx,   'Yanlış Sınıflandırılanlar')